In [1]:
import patient as p

import numpy as np
import myLib
import consts as c



In [2]:
c.WINDOW_SIZE = 55
c.POSITIVE_EXRTACT_END_OFFSET = 30
tabSegments =  []
for pStr in c.trainPatients:
    patient = p.patient(pStr)
    pos = patient.getPositveSegmentsAdvanced()
    neg = patient.getNegativesN(len(pos))
    tabSegments = tabSegments + pos + neg # add positves
    #tabSegments = tabSegments + patient.getNegativesN(len(tabSegments[-1][0]))
    


x_train,y_train = myLib.processDF(tabSegments)

x_test,y_test,x_validation,y_validation = myLib.getTrainValData()




In [3]:
#Sanity check

print(f"train data: {len(x_train)}\t validation: {len(x_validation)}\t test:{len(x_test)}\n")
unique, counts = np.unique(y_train, return_counts=True)

print(f"TRAIN:     Negatives: {counts[0]} Positives: {counts[1]}")
pos = counts[1]
neg = counts[0]
ratio = round(100 / len(x_train) * pos)
unique, counts = np.unique(y_validation, return_counts=True)
print(f"VALIDATION Negatives: {counts[0]} Positves: {counts[1]}")
unique, counts = np.unique(y_test, return_counts=True)
print(f"TEST       Negatives: {counts[0]} Positves: {counts[1]}")


print(f"\n{ratio} % of positives")

train data: 512	 validation: 24	 test:20

TRAIN:     Negatives: 256 Positives: 256
VALIDATION Negatives: 12 Positves: 12
TEST       Negatives: 10 Positves: 10

50 % of positives


In [4]:
import tensorflow as tf
METRICS = [
      #tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      #tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      #tf.keras.metrics.Precision(name='precision'),
      #tf.keras.metrics.Recall(name='recall'),
      #tf.keras.metrics.AUC(name='auc'),
      #tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def getModel2(metrics=METRICS, output_bias=None):
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(
          64, activation='relu',
          input_shape=(x_train.shape[-1],)),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(1, activation='sigmoid'),
  ])

  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model





In [7]:
model = getModel2()
from keras_visualizer import visualizer
visualizer(model, format='png', view=True)

TrainHistory = model.fit(
    x_train,
    y_train,
    epochs=c.EPOCHS,
    batch_size=c.BATCH_SIZE,
    validation_data=(x_validation, y_validation), 
    verbose=0)

In [ ]:
import matplotlib.pyplot as plt


def plot_metrics(history):
  metrics = ['loss', 'accuracy']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], color="C1", label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color="C0", linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])

    plt.legend()

plot_metrics(TrainHistory)
evalutation = model.evaluate(x_test,y_test,verbose=False)
print(f"Test accuracy: {round(evalutation[3],3)}")

In [ ]:

myLib.safeRun(model,"50/50 advanced",x_train,x_validation,x_test,y_test,"5SecBHT.csv")
